# Flight Data Recorder
<img src="media/fdr.jpg" width="300px" height="auto">

The flight data recorder is a microcontroller based system that uses a multitude of sensors to capture in-flight performance data. This includes GPS, altitude, heading, pitch, roll, angular acceleration, linear acceleration, temperature, and wind speed. The sensors are sampled at 100 Hz and written onto a microSD card in CSV format. The data recorder is removed after landing (or crash), and the data is retreived. In the case of a crash, the data visualizations can help piece together what went wrong, much like a real black box. Important metrics such as altitude and ground speed can simply be measured from the data. We can also apply descriptive analytics, to visualize patterns and correlations. This information can then be used by the team to help make design decisions and improve plane performance.

There are three useful notebooks: `flight_report.ipynb`, `data_analysis.ipynb`, and `demo.ipynb`. The `flight_report.ipynb` notebook is designed to quickly generate interactive flight reports from input data. Use this notebook if you just want to look at the reports. `data_analysis.ipynb` is better for understanding the code, as it is better documented and split into more digestable pieces. The last notebook, `demo.ipynb`, is more of a playground to get a feel for some useful python packages.

Flight Test Wednesday
Dolly
Flight plan:
1. Test sensor deployment and lights
2. "
3. "
4. 10 minute endurance
 - Flight pattern
5. Performance
 - Knife edge
 - Barrel roll
 - Vertical climb
 - Stall turn
 

### Install Packages

```
conda install -c conda-forge ipympl cartopy
```

## Setting up the data input:


In [13]:
import pandas as pd
from math import pi,sqrt,sin,cos,asin,atan2,radians

## Modify sensor_files, fdr_files to match the files you would like to be processed. 
## Make sure the order of files correspond to matching tests
#sensor_files = ['sensor_1.TXT', 'sensor_2.TXT', 'sensor_3.TXT','sensor_4.TXT','sensor_5.TXT','sensor_6.TXT']
fdr_files = ['FLIGHT01.TXT','FLIGHT02.TXT','FLIGHT03.TXT','FLIGHT04.TXT','FLIGHT05.TXT',]

# Use relative path (you shouldn't need to change this)
path = 'inputData(3-17-21)/'

# the following specifies the order of data coming in a line of the CSV file; do not modify
# Count, System Calibration level (0-3), Linear Acceleration XYZ (m/s^2), Gyro XYZ (radians/sec), Quaternion WXYZ
#sensor_header = ['count', 'sr_sys', 'sr_x_accel', 'sr_y_accel', 'sr_z_accel', 'sr_x_rps','sr_y_rps', 'sr_z_rps', 
#                 'sr_qw', 'sr_qx', 'sr_qy', 'sr_qz']

# the FDR collects the same info as the sensor, plus GPS data appended on the end in NMEA GPGGA sentences
fdr_header = ['count', 'fr_sys', 'fr_x_accel', 'fr_y_accel', 'fr_z_accel', 'fr_x_rps', 'fr_y_rps', 'fr_z_rps', 'fr_qw', 
              'fr_qx', 'fr_qy', 'fr_qz', 'gps', 'UTC', 'lat', 'NS', 'long', 'EW', 'fix', 'sats', 'HDOP', 'elev', 'units1', 
              'geoid', 'units2', 'age', 'checksum']

## Parsing the input data into dataframes:

The input data comes in as text files in CSV format. We need to get the data off disk and into a data structure that is easier to read. We'll use the panadas package, which provides powerful tools for data analysis. For each flight, we'll open its corresponding files from the filepath and store the data into a dataframe. In the end we'll have a list of dataframes `df_list`.

In [14]:
### Parse input data files into dataframe ###


# df_list stores each flight test as a dataframe
df_list = []

# puts the csv into dataframes with a header
for i in range(len(fdr_files)):
    # add sensor file
    #filepath = path + sensor_files[i]
    #df_sr = pd.read_csv(filepath, sep=",", header=None)
    # add header
    #df_sr.columns = sensor_header
    # add fdr file
    filepath = path + fdr_files[i]
    df_fr = pd.read_csv(filepath, sep=",", header=None)
    # add header
    df_fr.columns = fdr_header
    # merge dataframes sensor + fdr
    #df = pd.merge(df_sr, df_fr, how='left',on='count')
    df_list.append(df_fr)

## Defining some functions:

To get anything meaningful from the data, we'll need to make some functions that clean and convert it to more usable values. Also we'll do some calculations to extract other important information.

In [15]:
### Function Definitions: ###

### Cleaning Functions: ###


## removes uncalibrated rows from dataframe
def remove_rows(df):
    #rows = df[(df['sr_sys'] == 0) | (df['fr_sys'] == 0)].index
    #drop rows if sensor calibration level is 0
    rows = df[df['sr_sys'] == 0].index
    df.drop(rows, inplace=True)

### Conversion Functions: ###


## remove unused columns from dataframe
def remove_cols(df):
    df.drop(['gps','UTC','NS', 'EW','HDOP','units1', 'geoid', 'units2', 'age', 'checksum','fix','sats'], axis='columns', inplace=True)

## converts float from degree-minute-second to degree, latitude conversion
def dd_lat(x):
    degrees = int(x) // 100
    minutes = x - 100*degrees
    return degrees + minutes/60

## converts float from degree-minute-second to degree, longitude conversion
def dd_lon(x):
    return -dd_lat(x)


### Calculation Functions: ###

# Input row of dataframe
# returns bearing from prev row's to row's GPS coords
# https://stackoverflow.com/questions/54873868/python-calculate-bearing-between-two-lat-long
def get_bearing(row):
    lat1 = row['prev_lat']
    long1 = row['prev_long']
    lat2 = row['lat']
    long2 = row['long']
    dLon = (long2 - long1)
    x = cos(radians(lat2)) * sin(radians(dLon))
    y = cos(radians(lat1)) * sin(radians(lat2)) - sin(radians(lat1)) * cos(radians(lat2)) * cos(radians(dLon))
    bearing = atan2(x,y)
    return bearing

# input a row from dataframe
# returns the distance between prev coords to row's coords
# more precisely, it calculates the distance between two points on a sphere (ie. the earth)
# https://stackoverflow.com/questions/365826/calculate-distance-between-2-gps-coordinates
def haversine(row):
    lat1 = row['prev_lat']
    long1 = row['prev_long']
    lat2 = row['lat']
    long2 = row['long']                                                          
    degree_to_rad = float(pi / 180.0)
    d_lat = (lat2 - lat1) * degree_to_rad
    d_long = (long2 - long1) * degree_to_rad
    a = pow(sin(d_lat / 2), 2) + cos(lat1 * degree_to_rad) * cos(lat2 * degree_to_rad) * pow(sin(d_long / 2), 2)
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    mi = 3956 * c
    return mi

# input a row from dataframe
# returns the speed using distance/time
def speed(row):
    millis = row['count'] - row['prev_count']
    hours = millis / 36000
    return row['delta'] / hours

## Cleaning and processing the data:

Let's apply these functions to the dataframes!

In [16]:
### Clean data, Add some columns for calculations, Clean again ###

for i in range(len(df_list)):
    #remove_rows(df_list[i])
    remove_cols(df_list[i])
    # remove all rows with NaN values
    df_list[i].dropna(inplace=True)
    # apply latitude and longitude conversions to all rows
    df_list[i]['lat'] = df_list[i]['lat'].apply(dd_lat)
    df_list[i]['long'] = df_list[i]['long'].apply(dd_lon)
    # add prev_lat, prev_long, prev_count columns
    df_list[i]['prev_lat'] = df_list[i]['lat'].shift(1)
    df_list[i]['prev_long'] = df_list[i]['long'].shift(1)
    df_list[i]['prev_count'] = df_list[i]['count'].shift(1)
    # deletes first row
    df_list[i].dropna(inplace=True)
    # add a instantaneous bearing based off prev GPS coord
    df_list[i]['bearing'] = df_list[i].apply(lambda row: get_bearing(row), axis=1)
    # delta distance
    df_list[i]['delta'] = df_list[i].apply(lambda row: haversine(row), axis=1)
    # add mph column
    df_list[i]['mph'] = df_list[i].apply(lambda row: speed(row), axis=1)
    # remove temp. columns
    df_list[i].drop(columns=['delta', 'prev_lat', 'prev_long', 'prev_count'], inplace=True)

In [17]:
# remove bad mph readings
def remove_bad_rows(df):
    rows = df[df['mph'] > 75].index
    #drop rows if over 75 mph or equal 0
    rows1 = df[df['mph'] == 0].index
    df.drop(rows, inplace=True)
    df.drop(rows1, inplace=True)


for i in range(len(df_list)):
    remove_bad_rows(df_list[i])



In [18]:
print(len(df_list[0].columns))
df_list[0]

17


,count,fr_sys,fr_x_accel,fr_y_accel,fr_z_accel,fr_x_rps,fr_y_rps,fr_z_rps,fr_qw,fr_qx,fr_qy,fr_qz,lat,long,elev,bearing,mph
33,4933,0,0.02,0.02,-0.02,0.13,0.06,-0.06,0.0093,0.0029,0.0746,-0.9971,38.835383,-77.505783,68.4,1.570796,3.226970
62,4962,0,0.00,0.04,-0.03,-0.25,0.00,-0.06,0.0093,0.0029,0.0746,-0.9971,38.835385,-77.505783,68.4,0.000000,4.142714
124,5024,0,0.00,0.03,-0.04,0.00,0.06,0.00,0.0093,0.0029,0.0746,-0.9971,38.835387,-77.505783,68.4,0.000000,4.142714
146,5046,0,0.01,0.03,-0.01,0.00,0.00,0.00,0.0093,0.0029,0.0746,-0.9971,38.835387,-77.505782,68.4,1.570796,3.226970
167,5067,0,-0.02,0.04,-0.02,0.06,-0.06,0.00,0.0093,0.0027,0.0746,-0.9971,38.835388,-77.505782,68.4,0.000000,4.142714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3475,8377,3,0.51,0.59,0.07,-0.50,-0.06,0.19,-0.7845,0.0481,-0.1063,0.6089,38.835052,-77.506753,60.5,0.000000,4.142714
3490,8392,3,0.15,0.07,0.00,0.00,0.13,0.06,-0.7853,0.0524,-0.0787,0.6118,38.835052,-77.506752,60.4,1.570796,3.226985
3522,8424,3,0.05,0.04,-0.01,0.06,0.00,-0.13,-0.7854,0.0533,-0.0737,0.6123,38.835053,-77.506752,60.3,0.000000,4.142714
3551,8453,3,0.05,0.03,-0.01,-0.13,-0.13,0.13,-0.7854,0.0533,-0.0737,0.6123,38.835053,-77.506753,60.2,-1.570796,3.226985


# Time for Visualization!
## Select the Flight:
Select the flight number to do data visualization on. Use `flight_report.ipynb` to generate visualizations for all flights.

In [43]:
# index starts at 0
# input flights 0-4
f = 0

## Plotting the 2d flight path:
Now we can start visualizing the data. We'll start off with a 2d overhead view of the flight path. We'll use `matplotlib` and `cartopy` to project coordinates onto a 2d graph. We'll also import background tiles from Open Street Map, to give the path more context. We can also add some cool features such as a heatmap based on the speed and start/stop symbols.

In [44]:
### Lets plot the flight path over a map ###

# https://scitools.org.uk/cartopy/docs/latest/gallery/tube_stations.html#sphx-glr-gallery-tube-stations-py
#%matplotlib inline
%matplotlib widget
from matplotlib.path import Path
import matplotlib.pyplot as plt
import numpy as np
import cartopy.crs as ccrs
# uses map tiles from Open Street Map
from cartopy.io.img_tiles import OSM
from mpl_toolkits.axes_grid1 import make_axes_locatable

# add some padding
padding_lat = 0.002
padding_lon = 0.002
# set plot window coordinates (1: bottom left 2: top right)
lat1 = df_list[f]['lat'].min() - padding_lat
lon1 = df_list[f]['long'].min() - padding_lon
lat2 = df_list[f]['lat'].max() + padding_lat
lon2 = df_list[f]['long'].max() + padding_lon

imagery = OSM()
fig = plt.figure(figsize=(9, 9))
ax = fig.add_subplot(1, 1, 1, projection=imagery.crs)
ax.set_extent([lon1, lon2, lat1, lat2], ccrs.PlateCarree())

## Modify scale (15) to zoom in and out of the map (also increase/decreases pixelation)
ax.add_image(imagery, 15)

# plot all gps coords
scat = ax.scatter(df_list[f]['long'], df_list[f]['lat'],c=df_list[f]['mph'], cmap='autumn',s=10,alpha=1,transform=ccrs.PlateCarree())

# plot start and stop coords with special symbols
ax.scatter(df_list[f]['long'].head(1), df_list[i]['lat'].head(1),marker=">",color='green',edgecolors='black',linewidths=1,s=200,alpha=1,transform=ccrs.PlateCarree())
ax.scatter(df_list[f]['long'].tail(1), df_list[i]['lat'].tail(1),marker="s",color='red',edgecolors='black',linewidths=1,s=160,alpha=1,transform=ccrs.PlateCarree())

# add a colorbar
# https://stackoverflow.com/questions/30030328/correct-placement-of-colorbar-relative-to-geo-axes-cartopy
divider = make_axes_locatable(ax)
ax_cb = divider.new_horizontal(size="5%", pad=0.1, axes_class=plt.Axes)
fig.add_axes(ax_cb)
plt.colorbar(scat,cax=ax_cb,label='mph')

title = 'Flight #%s'%(f+1)
ax.set_title(title)
plt.show()
#fig.savefig("Test_route.png")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Plotting the 3d flight path:

In [45]:
### Now Lets plot the datapoints in 3D ###

%matplotlib widget
fig = plt.figure(figsize=(10,10)) 
ax = plt.axes(projection ='3d') 
ax.axes.set_xlim3d(left=df_list[f]['long'].min(), right=df_list[f]['long'].max()) 
ax.axes.set_ylim3d(bottom=df_list[f]['lat'].min(), top=df_list[f]['lat'].max()) 
ax.axes.set_zlim3d(bottom=0, top=df_list[f]['elev'].max()) 
ax.axes.set_xlabel('Longitude')
ax.axes.set_ylabel('Latitude')
ax.axes.set_zlabel('Elevation')
title = 'Flight #%s'%(f+1)
ax.set_title(title)

ax.scatter(df_list[f]['long'], df_list[f]['lat'], df_list[f]['elev'],s=10,c=df_list[f]['mph'],cmap='autumn',depthshade=True) 

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [46]:
%matplotlib widget

#df_list[f].plot.scatter(x='count', y='fr_x_accel', c='mph', cmap='autumn')
df_list[f].plot.scatter(x='count', y='mph', c='mph', cmap='autumn', title='Speed vs Time')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:title={'center':'Speed vs Time'}, xlabel='count', ylabel='mph'>

In [47]:
seconds = (df_list[f]['count'].max() - df_list[f]['count'].min())/10
minutes = int(seconds//60)
remainder = round(seconds%60,2)
print('Flight Time: ' + str(minutes) + ' min ' + str(remainder) + ' sec')
print('Avg Elevation: '+ str(round(df_list[f]['elev'].mean(),2)) + ' ft')
print('Max Elevation: ' + str(df_list[f]['elev'].max()) + ' ft')
print('Avg Speed: ' + str(round(df_list[f]['mph'].mean())) + ' mph')
print('Median Speed: ' + str(round(df_list[f]['mph'].median(),2)) + ' mph')
print('Max Speed: ' + str(round(df_list[f]['mph'].max())) + ' mph')
accel_axes = []
accel_axes.append(df_list[f]['fr_x_accel'].max()/9.8)
accel_axes.append(df_list[f]['fr_y_accel'].max()/9.8)
accel_axes.append(df_list[f]['fr_z_accel'].max()/9.8)
print("Max g's: " + str(round(max(accel_axes),2)))

Flight Time: 5 min 53.0 sec
Avg Elevation: 81.03 ft
Max Elevation: 115.2 ft
Avg Speed: 48 mph
Median Speed: 48.38 mph
Max Speed: 70 mph
Max g's: 2.63
